In [1]:
# Install required libraries

!pip install sentence-transformers faiss-cpu openai python-pptx


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 74.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 kB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.3/175.3 kB 19.2 MB/s eta 0:00:00


In [2]:
# Import necessary libraries

import faiss
import numpy as np
import time
import os

from sentence_transformers import SentenceTransformer
from pptx import Presentation
from openai import OpenAI


In [6]:
#upload ppt
from google.colab import files

uploaded = files.upload()  # select L2-Human.pptx
pptx_file = list(uploaded.keys())[0]
print("Uploaded file:", pptx_file)

Saving L2 - Human (1).pptx to L2 - Human (1).pptx
Uploaded file: L2 - Human (1).pptx


In [24]:
# Function to extract text from PPT file

def extract_text_from_ppt(ppt_path):
    presentation = Presentation(ppt_path)
    text = ""

    for slide in presentation.slides:
        for shape in slide.shapes:
            if hasattr(shape, "text"):
                text += shape.text + "\n"

    return text


# Provide your PPT file path
ppt_path = "L2 - Human (1).pptx"   # 🔥 CHANGE THIS

# Extract text
text = extract_text_from_ppt(ppt_path)

print("Text extraction complete.")
print(text[:500])  # Preview first 500 characters


Text extraction complete.
The Human

Ref: Human Computer Interaction- Alan Dix, Janet Finlay, Gregory D Abowd, Russell Beale
Human, one of the first component in HCI

Human, the user, is after all the one whom computer systems are designed to assist.

The requirement of user is therefore the priority.


Introduction
Human psychology (cognitive psychology)
Understand the capabilities and limitations
Find the things  that are difficult and impossible
How human perceive the world around them, how they store and process info


In [25]:
# Function to split text into smaller chunks

def chunk_text(text, chunk_size=500, overlap=50):
    chunks = []
    start = 0

    while start < len(text):
        end = start + chunk_size
        chunks.append(text[start:end])
        start = end - overlap

    return chunks


# Create chunks
chunks = chunk_text(text, chunk_size=500)

print("Total chunks created:", len(chunks))


Total chunks created: 124


In [26]:
# Load embedding model
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# Convert chunks into embeddings
chunk_embeddings = embedding_model.encode(chunks)

print("Embeddings created.")


Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Embeddings created.


In [27]:
# Get embedding dimension
dimension = chunk_embeddings.shape[1]

# Create FAISS index
index = faiss.IndexFlatL2(dimension)

# Add embeddings to index
index.add(np.array(chunk_embeddings))

print("FAISS index created and embeddings stored.")


FAISS index created and embeddings stored.


In [28]:
# Ask user question
query = input("Enter your question: ")

# Convert query into embedding
query_embedding = embedding_model.encode([query])

# Search top 3 similar chunks
k = 3
distances, indices = index.search(np.array(query_embedding), k)

# Retrieve actual text chunks
retrieved_chunks = [chunks[i] for i in indices[0]]

# Combine retrieved chunks into context
context = "\n\n".join(retrieved_chunks)

print("\nRetrieved Context:\n")
print(context)


Enter your question: Why is brightness important in HCI design?

Retrieved Context:

reen at low brightness looks darker because it emits less light.
The brain uses luminance information to judge how bright the screen appears.

Processing and Interpretation of stimulus
Importance of brightness in HCI
In HCI, designers must consider brightness so that interfaces are:
Easy to see
Comfortable for the eyes
Usable in different lighting conditions (daylight, night, indoors)
In HCI, understanding how users perceive brightness helps designers create interfaces that are clear, readable, 

gners create interfaces that are clear, readable, and comfortable to use.
Practical HCI examples
Adjustable brightness: Users can increase or decrease screen brightness based on their surroundings.
Dark mode: Uses darker backgrounds with lighter text to reduce eye strain, especially at night.
Good color contrast: Clear contrast between text and background improves readability.

Processing and Interpretation of

In [29]:
!pip install groq


In [30]:
from groq import Groq
import os

os.environ["GROQ_API_KEY"] = "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"   # 🔥 apni key yaha daalo

client = Groq()


In [31]:
response = client.chat.completions.create(
    model="llama-3.1-8b-instant",   # ✅ Correct model
    messages=[
        {"role": "user", "content": prompt}
    ]
)

print(response.choices[0].message.content)


Information not found


In [32]:
print("Retrieved Chunks:\n")
for i, chunk in enumerate(retrieved_chunks):
    print(f"\n--- Chunk {i+1} ---\n")
    print(chunk)


Retrieved Chunks:


--- Chunk 1 ---

reen at low brightness looks darker because it emits less light.
The brain uses luminance information to judge how bright the screen appears.

Processing and Interpretation of stimulus
Importance of brightness in HCI
In HCI, designers must consider brightness so that interfaces are:
Easy to see
Comfortable for the eyes
Usable in different lighting conditions (daylight, night, indoors)
In HCI, understanding how users perceive brightness helps designers create interfaces that are clear, readable, 

--- Chunk 2 ---

gners create interfaces that are clear, readable, and comfortable to use.
Practical HCI examples
Adjustable brightness: Users can increase or decrease screen brightness based on their surroundings.
Dark mode: Uses darker backgrounds with lighter text to reduce eye strain, especially at night.
Good color contrast: Clear contrast between text and background improves readability.

Processing and Interpretation of stimulus
A third factor that w